In [7]:
!pip install kaggle

# all import

In [42]:
import numpy as np
import pandas as pd
import os
import kagglehub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack

# get data

In [3]:
path = kagglehub.dataset_download("lakshmi25npathi/online-retail-dataset")

print("Path to dataset files:", path)

KaggleApiHTTPError: 403 Client Error.

You don't have permission to access resource at URL: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset. Please make sure you are authenticated if you are trying to access a private resource or a resource requiring consent.

# chenge data tipe

In [31]:
df = pd.read_excel('retail.xlsx')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [34]:
df.to_csv('retail.csv', index=False)

In [35]:
from IPython.display import FileLink


FileLink('retail.csv')

C:\Users\asus\Desktop\MoshinLerninHomeWork\class p\retail.csv

In [4]:
df = pd.read_csv('retail.csv')

# data cleaning

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   525461 non-null  int64  
 1   Invoice      525461 non-null  object 
 2   StockCode    525461 non-null  object 
 3   Description  522533 non-null  object 
 4   Quantity     525461 non-null  int64  
 5   InvoiceDate  525461 non-null  object 
 6   Price        525461 non-null  float64
 7   Customer ID  417534 non-null  float64
 8   Country      525461 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 36.1+ MB


In [6]:
print(df['Quantity'].unique())

[   12    48    24    10    18     3    16     4     2     6     8     1
    28    30    60    32    56     9    25    36    20   -12    -6    -4
   -24    -3    -2   576   288    27   -96    64   160  -240  -192     5
    -1    72    50    54   -50     7   800    11    58   192    96   144
   100   180   216    15   240    22   108    33    90    81   -81   -48
    17    -5    14    13    40    -8    -9    21   120   -44   230 -1043
  -117   200   168    19   280   128  -150   -18   -23    80   300   450
    61  -770  -720   480    -7    88    46   -31   -16   954  -954  -200
   504    45   467 -1012  4320  5184  4008   -25   600   505   500    23
    74    47   -89   432   252    75   184  -600  -504  -252  -246  -169
    31    38  -106   -36   360   107    49   -42    42    66   -10  -725
   222    34    52   270   -20   -30   -60  3000   400    70   936   -32
  -120    35  -126   -64  -168  -100   256  -108   -67  -360  1500    65
   -58   320   130    26   -45   -54  1002  1080   

In [63]:
print(df.isnull().sum())

Unnamed: 0          0
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [7]:
df = df.drop(columns=['Unnamed: 0','Invoice','Customer ID'])

## cheang data type

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   StockCode    525461 non-null  object 
 1   Description  522533 non-null  object 
 2   Quantity     525461 non-null  int64  
 3   InvoiceDate  525461 non-null  object 
 4   Price        525461 non-null  float64
 5   Country      525461 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 24.1+ MB


In [9]:
print(df['Price'].min())
print(df['Price'].max())

-53594.36
25111.09


In [10]:
df['Price'] = np.abs(df['Price'])

In [11]:
df['Quantity'] = df['Quantity'].astype('int16')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.to_period('M')
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['Price'] = df['Price'].astype('float32')


# mashin lerning

## count all word

In [12]:
df

,StockCode,Description,Quantity,InvoiceDate,Price,Country,InvoiceMonth
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12,6.95,United Kingdom,12
1,79323P,PINK CHERRY LIGHTS,12,2009-12,6.75,United Kingdom,12
2,79323W,WHITE CHERRY LIGHTS,12,2009-12,6.75,United Kingdom,12
3,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12,2.10,United Kingdom,12
4,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12,1.25,United Kingdom,12
...,...,...,...,...,...,...,...
525456,22271,FELTCRAFT DOLL ROSIE,2,2010-12,2.95,United Kingdom,12
525457,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12,3.75,United Kingdom,12
525458,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12,3.75,United Kingdom,12
525459,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12,3.75,United Kingdom,12


In [13]:
print(df['InvoiceMonth'].unique())

[12  1  2  3  4  5  6  7  8  9 10 11]


In [14]:
df['Country'] = df['Country'].str.replace(" ", "_")
df['Description'] = df['Description'].fillna('')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype    
---  ------        --------------   -----    
 0   StockCode     525461 non-null  object   
 1   Description   525461 non-null  object   
 2   Quantity      525461 non-null  int16    
 3   InvoiceDate   525461 non-null  period[M]
 4   Price         525461 non-null  float32  
 5   Country       525461 non-null  object   
 6   InvoiceMonth  525461 non-null  int64    
dtypes: float32(1), int16(1), int64(1), object(3), period[M](1)
memory usage: 23.1+ MB


In [16]:
df["MixedFeatures"] = df["Description"] + " " + df["Country"]
numeric_features = df[['Price', 'Quantity','InvoiceMonth']]
combined_features = hstack([, numeric_features])

# بخاطر کمبود حافظه مجبور به اینکار برایه تست شدم 

In [17]:
df_sample = df.sample(n=5000, random_state=42)


In [20]:
vectorizer = CountVectorizer(stop_words="english")
vector = vectorizer.fit_transform(df_sample["MixedFeatures"])
#vector = vectorizer.fit_transform(df["MixedFeatures"])
vectorizer.vocabulary_

{'red': 1063,
 'retrospot': 1075,
 'mini': 829,
 'cases': 250,
 'united_kingdom': 1368,
 'hot': 662,
 'baths': 111,
 'metal': 821,
 'sign': 1172,
 '72': 32,
 'cake': 217,
 'dolly': 434,
 'girl': 589,
 'design': 412,
 'harmonica': 634,
 'box': 177,
 'office': 894,
 'mug': 855,
 'warmer': 1386,
 'black': 149,
 'silver': 1174,
 'glass': 593,
 'heart': 640,
 'light': 758,
 'holder': 650,
 'ceramic': 258,
 'birdhouse': 142,
 'roof': 1091,
 'small': 1194,
 'blue': 159,
 'piece': 973,
 'polkadot': 1002,
 'cutlery': 391,
 'set': 1146,
 'butterfly': 207,
 'cookie': 347,
 'cutters': 392,
 'pink': 977,
 'fairy': 493,
 'childrens': 283,
 'apron': 65,
 'alphabet': 52,
 'stencil': 1239,
 'craft': 361,
 'vintage': 1380,
 'keepsake': 711,
 'flower': 530,
 'charlie': 265,
 'lola': 771,
 'personal': 965,
 'doorsign': 440,
 'feltcraft': 505,
 'doll': 433,
 'maria': 803,
 'belgium': 127,
 'multicolour': 859,
 '3d': 20,
 'balls': 95,
 'garland': 577,
 'stand': 1232,
 'hanging': 631,
 'cakes': 218,
 'mirror

## similarity

In [30]:
similarities = cosine_similarity(vector)


In [32]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 391453 to 497348
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype    
---  ------         --------------  -----    
 0   StockCode      5000 non-null   object   
 1   Description    5000 non-null   object   
 2   Quantity       5000 non-null   int16    
 3   InvoiceDate    5000 non-null   period[M]
 4   Price          5000 non-null   float32  
 5   Country        5000 non-null   object   
 6   InvoiceMonth   5000 non-null   int64    
 7   MixedFeatures  5000 non-null   object   
dtypes: float32(1), int16(1), int64(1), object(4), period[M](1)
memory usage: 302.7+ KB


In [33]:
df_sample

,StockCode,Description,Quantity,InvoiceDate,Price,Country,InvoiceMonth,MixedFeatures
391453,20750,RED RETROSPOT MINI CASES,6,2010-10,7.95,United_Kingdom,10,RED RETROSPOT MINI CASES United_Kingdom
440086,82583,HOT BATHS METAL SIGN,3,2010-11,2.10,United_Kingdom,11,HOT BATHS METAL SIGN United_Kingdom
369036,22951,72 CAKE CASES DOLLY GIRL DESIGN,48,2010-10,0.55,United_Kingdom,10,72 CAKE CASES DOLLY GIRL DESIGN United_Kingdom
232578,21915,RED HARMONICA IN BOX,12,2010-06,1.25,United_Kingdom,6,RED HARMONICA IN BOX United_Kingdom
41357,22311,OFFICE MUG WARMER BLACK+SILVER,12,2009-12,4.95,United_Kingdom,12,OFFICE MUG WARMER BLACK+SILVER United_Kingdom
...,...,...,...,...,...,...,...,...
489479,84378,SET OF 3 HEART COOKIE CUTTERS,3,2010-11,1.25,United_Kingdom,11,SET OF 3 HEART COOKIE CUTTERS United_Kingdom
319491,21931,JUMBO STORAGE BAG SUKI,10,2010-08,1.95,United_Kingdom,8,JUMBO STORAGE BAG SUKI United_Kingdom
517740,22385,JUMBO BAG SPACEBOY DESIGN,3,2010-12,1.95,United_Kingdom,12,JUMBO BAG SPACEBOY DESIGN United_Kingdom
184515,37503,TEA TIME CAKE STAND IN GIFT BOX,1,2010-05,21.23,United_Kingdom,5,TEA TIME CAKE STAND IN GIFT BOX United_Kingdom


In [40]:
df.columns

Index(['StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price',
       'Country', 'InvoiceMonth', 'MixedFeatures'],
      dtype='object')

In [41]:
pruduct = input("Enter pruduct name: ")


try:
    pruduct_index = df_sample.where(df_sample['StockCode'] == pruduct).dropna().index[0]

    movie_df = np.c_[df_sample['StockCode'], similarities]
    
    np.argsort(similarities[pruduct_index])
    similar_pruduct = list(np.argsort(similarities[pruduct_index]))
    similar_pruduct.reverse()
    n_recommendations = 20
    recommended_index = similar_pruduct[1:n_recommendations+1]
    df[['StockCode']].iloc[recommended_index]

except:
    print("pruduct is not in my list")
    

pruduct is not in my list


In [ ]:

# ۲. فیلتر کردن بر اساس معیار عددی (مثلاً قیمت)
base_price = df.iloc[item_index]['Price']
# در اینجا می‌توانید یک محدوده قیمتی مشخص کنید (مثلاً ۲۰ درصد اختلاف)
price_range_min = base_price * 0.8
price_range_max = base_price * 1.2

# پیدا کردن ایندکس‌هایی که در محدوده قیمتی قرار دارند
valid_indices = df[(df['Price'] >= price_range_min) & (df['Price'] <= price_range_max)].index

# ۳. ترکیب نتایج و مرتب‌سازی نهایی
# حالا باید امتیازات شباهت را برای ایندکس‌های معتبر (valid_indices) استخراج کنید
valid_similarities = similarities[0][valid_indices]

# مرتب‌سازی نتایج فیلتر شده بر اساس امتیاز شباهت
sorted_indices = valid_indices[np.argsort(valid_similarities)[::-1]]

# نمایش نتایج نهایی
recommended_items = df.iloc[sorted_indices].head(10)
print(recommended_items)